In [ ]:
# File: trapped.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import os
import tempfile

import numpy as np
import sdf
from frontend import App

# create an app
app = App.create("trapped")
filepath = os.path.join(tempfile.gettempdir(), "squishy.tmp.ply")

# create squishy ball mesh with spiky protrusions if not exists
if not os.path.exists(filepath):
    V, F = app.mesh.icosphere(r=2, subdiv_count=2)
    func = sdf.sphere(1.1)
    # add capsule spikes radiating outward from center
    for f in F:
        d = np.mean(V[f], axis=0)
        if d[0] > 0:
            func = func | sdf.capsule(-d, d, 0.05)
    func.save(filepath, step=0.03)

# load and process the squishy mesh
V, F, T = (
    app.mesh.load_tri(filepath)
    .decimate(100000)
    .tetrahedralize()
    .normalize()
    .scale(0.97)
)
app.asset.add.tet("squishy", V, F, T)

# create a scene
scene = app.scene.create()

# add invisible sphere container with narrow passages
(
    scene.add.invisible.sphere([0, 0, 0], 0.98)
    .invert()
    .radius(0.4, 2)
    .radius(0.4, 3)
    .radius(10, 4)
)

# add two squishy balls positioned to collide
scene.add("squishy").at(0.5, 0, 0).jitter()
scene.add("squishy").at(-0.5, 0, 0).jitter()

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters - disable gravity and increase sparse matrix capacity
(
    session.param.set("gravity", 0.0)
    .set("csrmat-max-nnz", 3000000)
    .set("dt", 0.01)
)

# set dynamic playback speed to slow down after initial collision
session.param.dyn("playback").time(2.99).hold().time(3).change(0.1)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()